In [1]:
# Imports
import IPython
import IPython.display
import pandas as pd
from numpy import mean
from numpy import std
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np
import glob
import tensorflow as tf
import math
import seaborn as sns
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

np.random.seed(37151)
tf.random.set_seed(37151)

In [2]:
time = 2.0
steps = int(time * 200)

In [3]:
class LSTMClassification():
    def __init__(self):
         self.df = pd.read_csv('../master_df_full.csv')     
    
    # model_output_size = numero de categorias a determinar
    # mag_offset = valor magnitudeminimo a categorizar
    def prep_features(self, model_output_size=3, mag_offset=4, timedelta = 400):
        self.timedelta = timedelta
        self.model_output_size = model_output_size
        features = []
        labels = []
        for index, row in self.df.iterrows():
            try:
                records = row['list_records'].replace("'", '')
                records = records.replace('[', '')
                records = records.replace(']', '')                                  
                list_records = records.split(', ')
                magnitude = row['Mag']
                for record in list_records:
                    df_ts = pd.read_csv("../Renamed_Records/" + record, index_col=None, header=0, dtype=np.float32)
                    features.append(df_ts.iloc[:timedelta].to_numpy(dtype='float'))
                    labels.append(np.eye(model_output_size)[magnitude-mag_offset])
            except:
                continue
        self.features=np.array(features)
        self.labels=np.array(labels)
    
    def split_dataset(self, split = 0.8):
        self.trainX, self.testX, self.trainY, self.testY = train_test_split(self.features, self.labels, test_size = split, shuffle = True)
        print(self.trainX)
        print(self.trainY)
    
    def create_model(self, height=-1, width=2, model_output_size=-1):
        model = Sequential()
        if height == -1:
            height = self.timedelta
            
        if model_output_size == -1:
            model_output_size = self.model_output_size

        # input shape = num timesteps, num features(temp, acc.)
        model.add(LSTM(100, input_shape = (height, width)))
        model.add(Dropout(0.5))
        model.add(Dense(100, activation = 'relu'))
        model.add(Dense(model_output_size, activation = 'softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        # Printing the models summary
        #model.summary()
        return model
    
    def evaluate_model(self, model, trainX, trainY, testX, testY, epochs=100, batch_size=20, verbose=0):
        history = model.fit(trainX, trainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
        loss, accuracy = model.evaluate(testX, testY, batch_size, verbose=verbose)
        return accuracy
    
    def predict_model(self, model, data):
        features = data['Values'].to_numpy().astype(float)
        predictions = model.predict(features)
        return predictions

In [4]:
# 2 segundos

classifier = LSTMClassification()
classifier.prep_features(model_output_size=5, mag_offset=3, timedelta = 2*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(2042, 400, 2)
(2042, 5)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  8.05575837e-08]
  [ 9.99999978e-03  1.61115167e-07]
  ...
  [ 1.98500001e+00 -1.69272894e-07]
  [ 1.99000001e+00 -7.15840770e-07]
  [ 1.99500000e+00  2.36778101e-06]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -2.05982673e-07]
  [ 9.99999978e-03 -4.11965345e-07]
  ...
  [ 1.98500001e+00 -5.26989334e-06]
  [ 1.99000001e+00 -2.90415187e-06]
  [ 1.99500000e+00 -2.53909349e-07]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  2.58600039e-06]
  [ 9.99999978e-03  5.17302033e-06]
  ...
  [ 1.98500001e+00 -1.68022211e-03]
  [ 1.99000001e+00 -1.64876389e-03]
  [ 1.99500000e+00 -1.57075759e-03]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -6.01601969e-05]
  [ 9.99999978e-03 -1.20481513e-04]
  ...
  [ 1.98500001e+00  9.25798900e-03]
  [ 1.99000001e+00 -1.57248299e-03]
  [ 1.99500000e+00 -1.38391564e-02]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -0.00000000e+00]
 

0.39228886365890503

In [5]:
# 4 segundos

classifier = LSTMClassification()
classifier.prep_features(model_output_size=5, mag_offset=3, timedelta = 4*200)
print(classifier.features.shape)
print(classifier.labels.shape)

classifier.split_dataset()
# height = numero de timestamps, width (time, acc.)
LSTM_magnitude_model = classifier.create_model()

# struct: (Number or files, number of steps, number of columns)
# (70, 400, 2)
classifier.evaluate_model(LSTM_magnitude_model, classifier.trainX,  classifier.trainY,  classifier.testX,  classifier.testY, verbose=1)

(2042, 800, 2)
(2042, 5)
[[[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  0.00000000e+00]
  [ 9.99999978e-03  1.01971620e-09]
  ...
  [ 3.98499990e+00  1.01971620e-09]
  [ 3.99000001e+00  0.00000000e+00]
  [ 3.99499989e+00  0.00000000e+00]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -3.16112008e-08]
  [ 9.99999978e-03 -6.32224015e-08]
  ...
  [ 3.98499990e+00 -8.30456884e-06]
  [ 3.99000001e+00  7.06663343e-07]
  [ 3.99499989e+00  6.67608219e-06]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  1.10129349e-07]
  [ 9.99999978e-03  2.19238984e-07]
  ...
  [ 3.98499990e+00  9.29328598e-05]
  [ 3.99000001e+00  8.40521461e-05]
  [ 3.99499989e+00  6.18060221e-05]]

 ...

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03 -3.05914871e-09]
  [ 9.99999978e-03 -6.11829742e-09]
  ...
  [ 3.98499990e+00  1.52039684e-06]
  [ 3.99000001e+00  8.29029261e-07]
  [ 3.99499989e+00  1.04724859e-06]]

 [[ 0.00000000e+00  0.00000000e+00]
  [ 4.99999989e-03  6.93407003e-08]
 

0.402080774307251